In [25]:
!pip install mlxtend

In [60]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [111]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/7105/Customer Preference Survey.csv')

In [112]:
df.drop(columns=['Timestamp'], inplace=True)
df.drop(index=0, inplace=True)
df.fillna("ไม่", inplace=True)

In [113]:
def encode_units(x):
    if "ไม่" in x:
        return 0
    else:
      return 1

hot_encoded_df = df.applymap(encode_units)

In [114]:
hot_encoded_df.head()

,playstation5,เครื่องทำขนมปัง,Ergonomic Wrist Rest,เครื่องอบผ้า,เครื่องชงกาแฟแคปซูล,เก้าอี้ LA-Z-Boy,เครื่องให้อาหารสัตว์อัตโนมัติ,บัตตาเลี่ยน,แก้วเก็บความเย็น,ลู่วิ่งออกกำลังกาย,Kindle,เครื่องซักผ้า,Bluetooth Speaker,ห้องน้ำแมวอัตโนมัติ,PS5,ทรายแมว,ลำโพง pixel,Logitech Mx Master 3 Mouse,ตุ๊กตา ty,น้ำพุแมว,Robot ดูดฝุ่น,Mechanical keyboard,Nintendo switch,หนังสือ python,gaming chair,Deskmat,Dew - ไฟโรเซ่,เทียนหอม jo malone,กระติกน้ำ 2 ลิตร,ที่นอน memory form,พลาสเตอร์บรรเทาปวด ตราเสือ,การ์ดจอ RTX 3080,ขนมจีนน้ำยาปู,Salmon Sashimi,จักรยานเสือหมอบ,ไฟแต่งห้องมินิมอล,External Harddisk,หม้อทอดไร้น้ํามัน,airpods,ยาดม,ไฟส่องหน้าไลฟ์สด
1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0
2,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0
3,0,0,0,1,1,0,0,1,1,1,0,1,1,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,1,1,1,0,1,1,0,1,1,1,1,1,1
4,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0
5,0,0,0,1,0,1,0,1,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,1,1,1,0,0,1,1,0,1,1,0,1,0


In [115]:
item_item_matrix = pd.DataFrame(index=hot_encoded_df.columns,columns=hot_encoded_df.columns)

In [116]:
from scipy.spatial.distance import cosine

for i in range(0,len(item_item_matrix.columns)) :
    # Loop through the columns for each column
    for j in range(0,len(item_item_matrix.columns)) :
      # Fill in placeholder with cosine similarities
      item_item_matrix.iloc[i,j] = 1 -cosine(hot_encoded_df.iloc[:,i],hot_encoded_df.iloc[:,j])

In [127]:
links = item_item_matrix.rename_axis('related item', 
                                     axis='columns').stack().reset_index()
links.columns = ['item', 'related item','value']

In [128]:
links_filtered.sort_values(by="value", ascending=False).head(20)

,item,related item,value
14,playstation5,PS5,1
574,PS5,playstation5,1
634,ทรายแมว,น้ำพุแมว,0.912871
794,น้ำพุแมว,ทรายแมว,0.912871
1488,External Harddisk,Bluetooth Speaker,0.789437
528,Bluetooth Speaker,External Harddisk,0.789437
1484,External Harddisk,แก้วเก็บความเย็น,0.772691
364,แก้วเก็บความเย็น,External Harddisk,0.772691
1513,External Harddisk,หม้อทอดไร้น้ํามัน,0.756235
1553,หม้อทอดไร้น้ํามัน,External Harddisk,0.756235


In [129]:
links_filtered=links.loc[ (links['value'] > 0.1) & 
                         (links['item'] != links['related item']) &
                         (links['item'] != 'ยาดม') & (links['related item'] != 'ยาดม') &
                         (links['item'] != 'Salmon Sashimi') & (links['related item'] != 'Salmon Sashimi') ]

In [130]:
links_filtered.sort_values(by="value", ascending=False).head(20)

,item,related item,value
14,playstation5,PS5,1
574,PS5,playstation5,1
634,ทรายแมว,น้ำพุแมว,0.912871
794,น้ำพุแมว,ทรายแมว,0.912871
1488,External Harddisk,Bluetooth Speaker,0.789437
528,Bluetooth Speaker,External Harddisk,0.789437
1484,External Harddisk,แก้วเก็บความเย็น,0.772691
364,แก้วเก็บความเย็น,External Harddisk,0.772691
1513,External Harddisk,หม้อทอดไร้น้ํามัน,0.756235
1553,หม้อทอดไร้น้ํามัน,External Harddisk,0.756235


ผมคิดว่าแต่ล่ะรายการไม่ได้เป็น basket แต่เป็นประวัติการซื้อของแต่ล่ะคนมากกว่าเลยทำเป็น collaborative filtering และตัดยาดม กับ Salmon Sashimi ออกเพราะคิดว่าเป็นสิ่งที่เกือบทุกคนใน class

Top 3 Recommendations
1. ทรายแมว กับ น้ำพุแมว
2. External Harddisk กับ Bluetooth Speaker
3. External Harddisk กับ แก้วเก็บความเย็น	